fichier original 

https://drive.google.com/file/d/1u4S5ILT6fCFk8J3Nkpyom9-pKjETI_u0/view?usp=drive_web

fichier augmenté



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import librosa
import IPython.display as ipd
from langdetect import detect
import csv
import os
import langid


## Loading audio files

In [ ]:
Audio_file = "C:\\Users\\angej\\Desktop\\Multilingual\\Beware_of_false_experts\\2020_03_25_000030-ci-covid19-unesco_msg03_eng.wav"


In [ ]:
# load audio files with librosa
Audio_file, sr = librosa.load(Audio_file)


## Basic information regarding audio files

In [ ]:
print(Audio_file.shape)

# duration in seconds of 1 sample
sample_duration = 1 / sr
print(f"One sample lasts for {sample_duration:6f} seconds")


## VISUALISATION DES CARACTERISTIQUES AUDIO

In [ ]:
# Créer un graphique
plt.figure(figsize=(10, 4))
plt.plot(Audio_file, label='Audio Signal')
plt.title('Audio Signal')
plt.ylabel('Amplitude')
plt.xlabel('Time')
plt.legend()
plt.show()

In [ ]:
# Créer un graphique
plt.figure(figsize=(10, 4))

# Définir la condition pour les amplitudes à colorer en rouge
red_condition = Audio_file > np.percentile(Audio_file, 99.50)  # Les 5% d'amplitudes les plus élevées

# Tracer les points de données en rouge si la condition est vraie
plt.plot(np.where(red_condition, Audio_file, np.nan), color='red', label='High Amplitude')

# Tracer les autres points de données en bleu
plt.plot(np.where(~red_condition, Audio_file, np.nan), label='Normal Amplitude')

plt.title('Audio Signal')
plt.ylabel('Amplitude')
plt.xlabel('Time')
plt.legend("2020_03_25_000030-ci-covid19-unesco_msg03_eng.wav")
plt.grid(True)

plt.show()

In [ ]:
# Obtenir le tempo et les beats
tempo, beat_frames = librosa.beat.beat_track(y=Audio_file, sr=sr)

# Obtenir le spectre de Mel
S = librosa.feature.melspectrogram(y=Audio_file, sr=sr, n_mels=128, fmax=8000)

# Visualiser le spectre de Mel
plt.figure(figsize=(10, 4))
S_dB = librosa.power_to_db(S, ref=np.max)
librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=sr, fmax=8000)
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogramme de Mel')
plt.tight_layout()
plt.show()

print(f'Tempo: {tempo}')
print(f'Beat frames: {beat_frames}')

In [ ]:
# Effectuer la STFT
D = librosa.stft(Audio_file)

# Convertir en amplitude
D_amp = librosa.amplitude_to_db(np.abs(D), ref=np.max)

# Visualiser l'analyse temporelle et fréquentielle
plt.figure(figsize=(15, 10))
librosa.display.specshow(D_amp, sr=sr, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogramme STFT')
plt.show()

In [ ]:
# Obtenir le ton chromatique
C = librosa.feature.chroma_cqt(y=Audio_file, sr=sr)

# Obtenir le contraste spectral
contrast = librosa.feature.spectral_contrast(S=S, sr=sr)

# Obtenir les statistiques
mean_S = np.mean(S)
std_S = np.std(S)
mean_C = np.mean(C)
std_C = np.std(C)
mean_contrast = np.mean(contrast)
std_contrast = np.std(contrast)

print(f'Standard deviation Mel spectrogram: {std_S}')
print(f'Mean Chroma: {mean_C}')
print(f'Standard deviation Chroma: {std_C}')
print(f'Mean Spectral contrast: {mean_contrast}')
print(f'Standard deviation Spectral contrast: {std_contrast}')

In [ ]:
# Créer une figure et des axes
fig, axs = plt.subplots(2, 1, figsize=(12, 8))

# Visualiser le ton chromatique
axs[0].bar(['Mean', 'Standard Deviation'], [mean_C, std_C])
axs[0].set_title('Chroma')

# Visualiser le contraste spectral
axs[1].bar(['Mean', 'Standard Deviation'], [mean_contrast, std_contrast])
axs[1].set_title('Spectral Contrast')


# Afficher la figure
plt.tight_layout()
plt.show()

In [ ]:
# Créer un DataFrame avec les mesures que vous avez calculées
df = pd.DataFrame({
    'Tempo': [tempo],
    'Mean Mel spectrogram': [mean_S],
    'Standard deviation Mel spectrogram': [std_S],
    'Mean Chroma': [mean_C],
    'Standard deviation Chroma': [std_C],
    'Mean Spectral contrast': [mean_contrast],
    'Standard deviation Spectral contrast': [std_contrast]
})

# Ajouter des colonnes pour le sexe et la langue
df['Transcription'] = ['']
df['Sexe'] = [''] 
df['Langue'] = ['']  

print(df)

## RECONNAISSANCE DE GENRE ET DE SEXE ET  TRANSCRIPTION


In [ ]:
import speech_recognition as sr
from pydub import AudioSegment


# Chemin pour sauvegarder le fichier CSV
csv_file = df = "C:\\Users\\angej\\Desktop\\Multilingual\\Code\\dataframe.csv"

# Créer un recognizer de la librairie speech_recognition
r = sr.Recognizer()

# AudioFile pour chaque fichier audio
audio_files = [#'C:\\Users\\angej\\Desktop\\Multilingual\\COVID_19_vaccine_side_effects_to_expect\\03_covid_19_vaccine_side_effects_to_expect_en.mp3'
               'C:\\Users\\angej\\Desktop\\Multilingual\\Gauge your emotional reaction\\2020-03-25-000040-ci-covid19-unesco_msg04_eng.wav']

# Créer une liste pour stocker les nouvelles données
nouvelles_donnees = []

for audio_file in audio_files:
    with sr.AudioFile(audio_file) as source:
        audio_data = r.record(source)
        text = r.recognize_google(audio_data)
        
        # Utiliser langid pour déterminer la langue
        lang, _ = langid.classify(text)
        
        # Ajouter les nouvelles données à la liste
        nouvelles_donnees.append({'Audio': audio_file,
                                  'Transcription': text,
                                  'Langue': lang ,
                                  'Tempo': tempo,
                                  'Mean Mel spectrogram':mean_S,'Standard deviation Mel spectrogram': std_S,
                                  'Mean Chroma':mean_C})
try:
    with open(csv_file, 'r') as file:
        # Vérifier si le fichier CSV existe déjà
        if os.path.isfile(csv_file):
            # Charger le DataFrame depuis le fichier CSV
            df = pd.read_csv(csv_file)
            
            # Ajouter les nouvelles données au DataFrame existant
            if not df.empty:
                # Convertir les nouvelles données en un DataFrame
                nouvelles_donnees_df = pd.DataFrame(nouvelles_donnees)
                
                # Ajouter les nouvelles données au DataFrame existant
                df = pd.concat([df, nouvelles_donnees_df], ignore_index=True, axis=0)
            else:
                # Créer un DataFrame avec les nouvelles données si le DataFrame est vide
                df = pd.DataFrame(nouvelles_donnees)
        else:
            # Créer un DataFrame avec les nouvelles données si le fichier n'existe pas
            df = pd.DataFrame(nouvelles_donnees)
except Exception as e:
    print("Une erreur est survenue lors de la lecture du fichier CSV :", e)

# Sauvegarder le DataFrame complet dans un fichier CSV
df.to_csv(csv_file, index=False)

# Afficher le DataFrame
print(df)


### DATA AUGMENTATIONS

In [ ]:
def plot_time_series(data):
    fig = plt.figure(figsize=(14, 8))
    plt.title('Audio Signal ')
    plt.ylabel('Amplitude')
    plt.plot(np.linspace(0, 1, len(data)), data)
    plt.show()
    
# Adding white noise 
wn = np.random.randn(len(Audio_file))
data_wn = Audio_file + 0.005*wn
plot_time_series(data_wn)
# We limited the amplitude of the noise so we can still hear the word even with the noise, 
#which is the objective
ipd.Audio(data_wn, rate=16000)

In [ ]:
# Shifting the sound
data_roll = np.roll(Audio_file, 1600)
plot_time_series(data_roll)
ipd.Audio(data_roll, rate=16000)

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd


# Function to apply pitch speed variation
def pitch_speed_variation(data, speed_factor):
    return librosa.effects.pitch_shift(data, sr=16000, n_steps=speed_factor)

# Function to apply noise injection
def noise_injection(data, noise_level=0.5):
    noise = np.random.normal(0, noise_level, len(data))
    return data + noise

# Function to apply time stretching
def time_stretch(data, rate=1.0):
    return librosa.effects.time_stretch(data, rate=rate)

# Function to apply controlled pitch variation
def controlled_pitch_variation(data, pitch_steps):
    return librosa.effects.pitch_shift(data, sr=16000, n_steps=pitch_steps)

# Apply augmentations
pitch_speed_data = pitch_speed_variation(Audio_file, 0.2)  # 20% increase in pitch speed
noisy_data = noise_injection(Audio_file, noise_level=0.5)
time_stretched_data = time_stretch(Audio_file, rate=0.6)  # 20% time stretching
controlled_pitch_data = controlled_pitch_variation(Audio_file, pitch_steps=2)

# Plot and play the original and augmented signals
plt.figure(figsize=(14, 5))
plt.subplot(2, 3, 1)
plt.plot(Audio_file)
plt.title('Original Audio')

plt.subplot(2, 3, 2)
plt.plot(pitch_speed_data)
plt.title('Pitch Speed Variation')

plt.subplot(2, 3, 3)
plt.plot(noisy_data)
plt.title('Noise Injection')

plt.subplot(2, 3, 4)
plt.plot(time_stretched_data)
plt.title('Time Stretching')

plt.subplot(2, 3, 5)
plt.plot(controlled_pitch_data)
plt.title('Controlled Pitch Variation')

plt.show()

ipd.Audio(Audio_file, rate=16000)
ipd.Audio(pitch_speed_data, rate=16000)
ipd.Audio(noisy_data, rate=16000)
ipd.Audio(time_stretched_data, rate=16000)
ipd.Audio(controlled_pitch_data, rate=16000)


In [ ]:
import librosa
import librosa.display
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import IPython.display as ipd

# Set Seaborn style
sns.set(style="whitegrid")

# Function to visualize the waveform using Seaborn and Matplotlib
def visualize_waveform(data, sr=16000, title="Waveform"):
    plt.figure(figsize=(14, 5))
    sns.lineplot(x=np.arange(len(data)) / sr, y=data)
    plt.title(title)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.show()

# Function to visualize the spectrogram using Seaborn and Matplotlib
def visualize_spectrogram(data, sr=16000, title="Spectrogram"):
    plt.figure(figsize=(14, 5))
    D = librosa.amplitude_to_db(np.abs(librosa.stft(data)), ref=np.max)
    librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log', cmap='viridis')
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.show()

# Function to visualize the frequency distribution using Seaborn and Matplotlib
def visualize_frequency_distribution(data, sr=16000, title="Frequency Distribution"):
    plt.figure(figsize=(14, 5))
    frequencies, amplitudes = librosa.magphase(librosa.stft(data))
    plt.plot(frequencies, np.log1p(amplitudes.mean(axis=1)))  # Use mean(axis=1) to collapse the second dimension
    plt.title(title)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Log Amplitude')
    plt.show()



# Function to visualize the Mel Spectrogram using Seaborn and Matplotlib
def visualize_melspectrogram(data, sr=16000, title="Mel Spectrogram"):
    plt.figure(figsize=(14, 5))
    mel_spec = librosa.feature.melspectrogram(y=data, sr=sr)  # Pass the audio data as 'y'
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    librosa.display.specshow(mel_spec_db, sr=sr, x_axis='time', y_axis='mel', cmap='viridis')
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.show()

# Function to visualize the Mel-Frequency Distribution using Seaborn and Matplotlib
def visualize_melfrequency(data, sr=16000, title="Mel-Frequency Distribution"):
    plt.figure(figsize=(14, 5))
    mel_filters = librosa.filters.mel(sr=sr, n_fft=2048, n_mels=128, fmin=0.0, fmax=None, htk=False)
    librosa.display.specshow(mel_filters, sr=sr, x_axis='linear', cmap='viridis')
    plt.title(title)
    plt.xlabel('Mel Bin')
    plt.ylabel('Amplitude (dB)')
    plt.colorbar(format='%+2.0f dB')
    plt.show()



# Example usage:

# Load your audio file or use a random signal as an example
# Example: audio_data, sr = librosa.load('your_audio_file.wav', sr=16000)
# Make sure to load the audio file with the correct sample rate (sr)
audio_data=Audio_file

# Visualize waveform using Seaborn
visualize_waveform(audio_data, sr=16000, title="Waveform")

# Visualize spectrogram using Seaborn
visualize_spectrogram(audio_data, sr=16000, title="Spectrogram")
# Visualize frequency distribution using Seaborn
visualize_frequency_distribution(audio_data, sr=16000, title="Frequency Distribution")
# Visualize Mel Spectrogram using Seaborn
visualize_melspectrogram(audio_data, sr=16000, title="Mel Spectrogram")

# Visualize Mel-Frequency Distribution using Seaborn
visualize_melfrequency(audio_data, sr=16000, title="Mel-Frequency Distribution")
